In [2]:
import numpy as np
import pandas as pd
import io
import re

In [4]:
column_names_ratings = ['UserID', 'MovieID', 'Rating', 'Timestamp']
column_names_movies = ['MovieID', 'Title', 'Genres']
ratings_df = pd.read_csv('ratings.dat', delimiter='::',names=column_names_ratings)
movies_df=pd.read_csv('movies.dat', delimiter='::',names=column_names_movies,encoding='latin1')

C:\Users\20109\AppData\Local\Temp\ipykernel_19324\3738254216.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings_df = pd.read_csv('ratings.dat', delimiter='::',names=column_names_ratings)
C:\Users\20109\AppData\Local\Temp\ipykernel_19324\3738254216.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_df=pd.read_csv('movies.dat', delimiter='::',names=column_names_movies,encoding='latin1')


In [19]:
def recommend(user_id,Movie_list=[]):
    
    df=ratings_df.copy()

    my_user=df[df['UserID']==user_id]

    if(Movie_list!=[]):
        my_user_latest_Movies=Movie_list

    my_user_latest_Movies=my_user.sort_values(by='Rating',ascending=False).head(5)

    similar_users=df[df['MovieID'].isin(my_user_latest_Movies['MovieID'].tolist()) &  (df['UserID'] != user_id) &(df['Rating'] > 4) ]

    similar_users=df[df['UserID'].isin(similar_users['UserID'])]

    similar_users_Movies=similar_users[~similar_users['MovieID'].isin(my_user_latest_Movies['MovieID'].tolist())]

    Movie_options=similar_users_Movies[similar_users_Movies['Rating'] >= 4]

    Movie_options=Movie_options.sort_values(by='Rating',ascending=False)

    new_df=Movie_options

    mean_ratings = Movie_options.groupby('MovieID')['Rating'].mean()

# Merge the mean_ratings back into the original DataFrame
    new_df = new_df.merge(mean_ratings, left_on='MovieID', right_index=True, suffixes=('', '_mean'))

    value_counts=new_df['MovieID'].value_counts()

    new_df['value_counts_col']=new_df['MovieID'].map(value_counts)

    new_df=new_df.sort_values(by=['value_counts_col','Rating_mean'],ascending=False)

    new_df=new_df.drop_duplicates(subset=['MovieID'])
    new_df=new_df.drop(columns=['UserID'])
    return new_df[:5]


In [20]:
print(recommend(16, [1724,200]).head())

        MovieID  Rating  Timestamp  Rating_mean  value_counts_col
563284        1       5  967170089     4.755700               307
557234     1196       5  967359892     4.736667               300
557190      260       5  967360233     4.854237               295
557253     3114       5  967364570     4.727586               290
557205     2858       5  967360479     4.773519               287
